In [131]:
commands = {
	# Ethernet switch commands and responses
    '0001': 'FEB_RD_SR',
	'0002' : 'FEB_WR_SR',
	'0003' : 'FEB_RD_SR_SRFF',
	'0004' : 'FEB_WR_SR_SRFF',
	'0000' : 'FEB_OK_SR',
	'00ff' : 'FEB_ERR_SR',

	# Board configuration and data acquisition
	'0101' : 'FEB_SET_RECV',
	'0102' : 'FEB_GEN_INIT',
	'0103' : 'FEB_GEN_HVON',
	'0104' : 'FEB_GEN_HVOFF',
	'0105' : 'FEB_GET_RATE',
	'0100' : 'FEB_OK',
	'01FF' : 'FEB_ERR',

	# CITIROC slow control register
	'0201' : 'FEB_RD_SCR',
	'0202' : 'FEB_WR_SCR',
	'0200' : 'FEB_OK_SCR',
	'02FF' : 'FEB_ERR_SCR',

	# CITIROC data control register
	'0301' : 'FEB_RD_CDR',
	'0300' : 'FEB_DATA_CDR',
	'0303' : 'FEB_EOF_CDR', 
	'03FF' : 'FEB_ERR_CDR',

	# CITIROC probe register
	'0401' : 'FEB_RD_PMR',
	'0402' : 'FEB_WR_PMR',
	'0400' : 'FEB_OK_PMR',
	'04ff' : 'FEB_ERR_PMR',

	# Firmware read-write transmission
	'0501' : 'FEB_RD_FW',
	'0502' : 'FEB_WR_FW',
	'0500' : 'FEB_OK_FW',
	'05ff' : 'FEB_ERR_FW',
	'0503' : 'FEB_EOF_FW',
	'0504' : 'FEB_DATA_FW',

	# FPGA input logic configuration register
	'0601' : 'FEB_RD_FIL',
	'0602' : 'FEB_WR_FIL',
	'0600' : 'FEB_OK_FIL',
	'06ff' : 'FEB_ERR_FIL',
}

In [164]:
def decode_command(packet):
    data = bytearray(packet.load)
    hex_string = ''.join(struct.pack('B', x).hex() for x in data)
    data_hex = []
    for i in range(length):
        data_hex.append(hex_string[(i*4):(i*4)+2])
        data_hex.append(hex_string[(i*4)+2:(i*4)+4])
    cmd_hex = ''.join(data_hex[1::-1])
    cmd = commands[cmd_hex]
    return cmd, data_hex[2:]

In [202]:
def print_packet(index, packet):
    origin = packet.src
    dst = packet.dst
    if packet.src == '00:60:37:12:34:45':
        origin = 'card'
        dst = 'host'
    if packet.src == '00:0a:f7:4d:4c:5f':
        origin = 'host'
        dst = 'card'
    if packet.src not in ['00:0a:f7:4d:4c:5f', '00:60:37:12:34:45']:
        origin = packet.src
    if packet.dst not in ['00:0a:f7:4d:4c:5f', '00:60:37:12:34:45']:
        dst = packet.dst[-2:]
    cmd, payload = decode_command(packet)
    result = '{:04} - {} -> {} \t {:04x} \t | {} \t | {} || {}'.format(index, origin, dst, packet.type, cmd.ljust(16), ''.join(payload[0:2]), ''.join(payload[2:16]))
    print(result)

In [204]:
from scapy.all import *

scapy_cap = rdpcap('/home/jmbenlloch/muons_test.pcapng')
for index, packet in enumerate(scapy_cap):
    print_packet(index+1, packet)

0001 - host -> ff 	 0801 	 | FEB_GEN_INIT     	 | 0100 || 000af74d4c5f0000000000000000
0002 - card -> host 	 0801 	 | FEB_OK           	 | 0000 || 000af74d4c5f0000000000000000
0003 - host -> ff 	 0801 	 | FEB_GEN_INIT     	 | 0200 || 000af74d4c5f0000000000000000
0004 - card -> host 	 0801 	 | FEB_OK           	 | 0000 || 000af74d4c5f0000000000000000
0005 - host -> card 	 0801 	 | FEB_GET_RATE     	 | 0000 || 000af74d4c5f0000000000000000
0006 - card -> host 	 0801 	 | FEB_OK           	 | 0000 || 000000004c5f0000000000000000
0007 - host -> card 	 0801 	 | FEB_RD_CDR       	 | 0000 || 000000004c5f0000000000000000
0008 - card -> host 	 0801 	 | FEB_DATA_CDR     	 | 1100 || 0000000021e30880292e38530000
0009 - card -> host 	 0801 	 | FEB_DATA_CDR     	 | 0000 || 0000000061e308809d108b510000
0010 - card -> host 	 0801 	 | FEB_EOF_CDR      	 | 0000 || 000000004c5f0000000000000000
0011 - host -> card 	 0801 	 | FEB_GET_RATE     	 | 0000 || 000000004c5f0000000000000000
0012 - card -> host 	 080